In [45]:
import pandas as pd
import numpy as np
from malnis import show
from sklearn.metrics import average_precision_score, roc_auc_score
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

In [27]:
data = pd.read_pickle("quotes.pkl")
show(data, 30)

(23, 3)


,text,query,relevance
0,"[29, R425–R473 Forum Using Soundscapes to Asse...",Hydrothermal vents are unique deep-sea structu...,"[0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]"
1,"[Therefore, using daily SEL to describe sounds...",Hydrothermal vents are unique deep-sea structu...,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ..."
2,[TECHNOLOGY AND CODE published: 19 August 2021...,Hydrothermal vents are unique deep-sea structu...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,[ORIGINAL RESEARCH published: 05 August 2021 d...,Hydrothermal vents are unique deep-sea structu...,"[0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, ..."
4,[A brief overview of current approaches for un...,Hydrothermal vents are unique deep-sea structu...,"[0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, ..."
5,[Due to its tremendous advantages this technol...,Cloud computing provides a shared pool of comp...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]"
6,[Interpretable & Explorable Approximations of ...,Prognostic modelling using machine learning te...,"[1, 0, 1, 0, 1, 0]"
7,"[Introduction Over the past decade, there has ...",Prognostic modelling using machine learning te...,"[1, 1, 1, 0, 1, 1, 0, 0]"
9,"[To gain insight into the black-box model, we ...",Prognostic modelling using machine learning te...,"[1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1]"
11,"[Next, three techniques, Bayesian networks, Lo...",Obesity is a significant problem in population...,"[1, 1, 1, 0, 0, 0, 1, 1, 0]"


In [14]:
train = pd.read_pickle("/home/jarobyte/scratch/malnis_dataset/data/data_train.pkl")
show(train)

(7172, 9)


,query,document,summary,r1,r2,rl,sentences,relevance,original_sentences
7593,Due to the lack of structured knowledge applie...,"Index Terms—Machine learning, knowledge discov...","[Text classification [226], text clustering [2...",0.278846,0.095941,0.259615,"[Index Terms—Machine learning, knowledge disco...","[False, False, False, False, False, False, Fal...",685
7554,Obtaining enough labeled data to robustly trai...,One of the greatest roadblocks to using modern...,"[“Lawyer”); moreover, these sources might be c...",0.295455,0.104247,0.272727,[One of the greatest roadblocks to using moder...,"[True, False, True, False, False, False, False...",246
4892,"Word embeddings, i.e., low-dimensional vector ...",Recently some studies have shown that text cla...,"[A recent work (Schuster et al., 2020) showed ...",0.407018,0.186364,0.385965,[Recently some studies have shown that text cl...,"[False, False, True, False, False, False, Fals...",202
4169,"In this paper our objectives are, first, netwo...",It is commonsense that how you look at an obje...,"[Views are commonly different sensory signals,...",0.257143,0.090909,0.250000,[It is commonsense that how you look at an obj...,"[False, False, False, False, False, False, Fal...",240
6690,"This paper describes TextTiling, an algorithm ...",Accurately representing the distance between t...,[Most similar to our method is that of Wan (20...,0.206349,0.037975,0.158730,[Accurately representing the distance between ...,"[False, False, False, False, False, False, Fal...",246


In [43]:
results = []

# classical models

In [15]:
%time
corpus = train.sentences.sum()
len(corpus)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


2089747

## words

In [16]:
word_vectorizer = TfidfVectorizer(
    analyzer = "word",
    ngram_range = (1, 1)
)
word_vectorizer.fit(corpus)

TfidfVectorizer()

In [28]:
word_features = data.text.map(word_vectorizer.transform)
len(word_features)

23

In [29]:
word_queries = data["query"].map(lambda x: word_vectorizer.transform([x]))
len(word_queries)

23

### tfidf

In [30]:
word_scores = [(f @ q.T).toarray().squeeze() for f, q in zip(word_features, word_queries)]
len(word_scores)

23

In [31]:
word_scores[0]

array([0.22349912, 0.26112057, 0.46502043, 0.04701891, 0.05946417,
       0.32419993, 0.16290082, 0.30282736, 0.2951947 , 0.17113323,
       0.27721748, 0.31616401, 0.39206688, 0.05603763, 0.07432639])

In [34]:
word_ap = [average_precision_score(l, s) for l, s in zip(tqdm(data.relevance), word_scores)]
len(word_ap)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [35]:
word_ra = [roc_auc_score(l, s) for l, s in zip(tqdm(data.relevance), word_scores)]
len(word_ra)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [44]:
results.append(
    dict(
        model = "tfidf_words",
        ap_mean = np.mean(word_ap), 
        ap_std = 2 * np.std(word_ap)/np.sqrt(len(word_ap)),
        ra_mean = np.mean(word_ra), 
        ra_std = 2 * np.std(word_ra)/np.sqrt(len(word_ra))
    )
)
results

[{'model': 'tfidf_words',
  'ap_mean': 0.5753637004394068,
  'ap_std': 0.11248102455922193,
  'ra_mean': 0.5425925714488694,
  'ra_std': 0.1312144617891538}]

## mlp

In [46]:
with open("/home/jarobyte/scratch/malnis_dataset/mlp_tfidf/words/models/15010241_7.pkl", "rb") as file:
    mlp_words = pickle.load(file)

In [51]:
word_mlp_scores = [mlp_words.predict_proba(x)[:, 1] for x in word_features]
len(word_mlp_scores)

23

In [52]:
word_mlp_ap = [average_precision_score(l, s) for l, s in zip(tqdm(data.relevance), word_mlp_scores)]
len(word_mlp_ap)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [53]:
word_mlp_ra = [roc_auc_score(l, s) for l, s in zip(tqdm(data.relevance), word_mlp_scores)]
len(word_mlp_ra)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [54]:
results.append(
    dict(
        model = "mlp_words",
        ap_mean = np.mean(word_mlp_ap), 
        ap_std = 2 * np.std(word_mlp_ap)/np.sqrt(len(word_mlp_ap)),
        ra_mean = np.mean(word_mlp_ra), 
        ra_std = 2 * np.std(word_mlp_ra)/np.sqrt(len(word_mlp_ra))
    )
)
results

[{'model': 'tfidf_words',
  'ap_mean': 0.5753637004394068,
  'ap_std': 0.11248102455922193,
  'ra_mean': 0.5425925714488694,
  'ra_std': 0.1312144617891538},
 {'model': 'mlp_words',
  'ap_mean': 0.6542930633091931,
  'ap_std': 0.12689653830649794,
  'ra_mean': 0.6504883219756941,
  'ra_std': 0.13484475623274275}]

## chars

In [55]:
char_vectorizer = TfidfVectorizer(
    analyzer = "char",
    ngram_range = (3, 3)
)
char_vectorizer.fit(corpus)

TfidfVectorizer(analyzer='char', ngram_range=(3, 3))

In [56]:
char_features = data.text.map(char_vectorizer.transform)
len(char_features)

23

In [57]:
char_queries = data["query"].map(lambda x: char_vectorizer.transform([x]))
len(char_queries)

23

### tfidf

In [58]:
char_scores = [(f @ q.T).toarray().squeeze() for f, q in zip(char_features, char_queries)]
len(char_scores)

23

In [59]:
char_scores[0]

array([0.27963787, 0.2435618 , 0.29566185, 0.36901626, 0.18608043,
       0.39333694, 0.28094598, 0.29278575, 0.24565227, 0.28015445,
       0.20125509, 0.37141184, 0.23642442, 0.08147654, 0.09050535])

In [60]:
char_ap = [average_precision_score(l, s) for l, s in zip(tqdm(data.relevance), char_scores)]
len(char_ap)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [61]:
char_ra = [roc_auc_score(l, s) for l, s in zip(tqdm(data.relevance), char_scores)]
len(char_ra)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [62]:
results.append(
    dict(
        model = "tfidf_chars",
        ap_mean = np.mean(char_ap), 
        ap_std = 2 * np.std(char_ap)/np.sqrt(len(char_ap)),
        ra_mean = np.mean(char_ra), 
        ra_std = 2 * np.std(char_ra)/np.sqrt(len(char_ra))
    )
)
results

[{'model': 'tfidf_words',
  'ap_mean': 0.5753637004394068,
  'ap_std': 0.11248102455922193,
  'ra_mean': 0.5425925714488694,
  'ra_std': 0.1312144617891538},
 {'model': 'mlp_words',
  'ap_mean': 0.6542930633091931,
  'ap_std': 0.12689653830649794,
  'ra_mean': 0.6504883219756941,
  'ra_std': 0.13484475623274275},
 {'model': 'tfidf_chars',
  'ap_mean': 0.6743565420052174,
  'ap_std': 0.10458764962687674,
  'ra_mean': 0.6343831537885246,
  'ra_std': 0.13090523765556375}]

## mlp

In [63]:
with open("/home/jarobyte/scratch/malnis_dataset/mlp_tfidf/chars/models/14986643_2.pkl", "rb") as file:
    mlp_chars = pickle.load(file)

In [64]:
char_mlp_scores = [mlp_chars.predict_proba(x)[:, 1] for x in char_features]
len(char_mlp_scores)

23

In [65]:
char_mlp_ap = [average_precision_score(l, s) for l, s in zip(tqdm(data.relevance), char_mlp_scores)]
len(char_mlp_ap)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [66]:
char_mlp_ra = [roc_auc_score(l, s) for l, s in zip(tqdm(data.relevance), char_mlp_scores)]
len(char_mlp_ra)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [67]:
results.append(
    dict(
        model = "mlp_chars",
        ap_mean = np.mean(char_mlp_ap), 
        ap_std = 2 * np.std(char_mlp_ap)/np.sqrt(len(char_mlp_ap)),
        ra_mean = np.mean(char_mlp_ra), 
        ra_std = 2 * np.std(char_mlp_ra)/np.sqrt(len(char_mlp_ra))
    )
)
results

[{'model': 'tfidf_words',
  'ap_mean': 0.5753637004394068,
  'ap_std': 0.11248102455922193,
  'ra_mean': 0.5425925714488694,
  'ra_std': 0.1312144617891538},
 {'model': 'mlp_words',
  'ap_mean': 0.6542930633091931,
  'ap_std': 0.12689653830649794,
  'ra_mean': 0.6504883219756941,
  'ra_std': 0.13484475623274275},
 {'model': 'tfidf_chars',
  'ap_mean': 0.6743565420052174,
  'ap_std': 0.10458764962687674,
  'ra_mean': 0.6343831537885246,
  'ra_std': 0.13090523765556375},
 {'model': 'mlp_chars',
  'ap_mean': 0.6644389816679619,
  'ap_std': 0.13069853702216494,
  'ra_mean': 0.6819279927056462,
  'ra_std': 0.1151384827842618}]

In [68]:
results_df = pd.DataFrame(results)
show(results_df)

(4, 5)


,model,ap_mean,ap_std,ra_mean,ra_std
0,tfidf_words,0.575364,0.112481,0.542593,0.131214
1,mlp_words,0.654293,0.126897,0.650488,0.134845
2,tfidf_chars,0.674357,0.104588,0.634383,0.130905
3,mlp_chars,0.664439,0.130699,0.681928,0.115138


In [73]:
print(results_df.round(3).to_latex())

\begin{tabular}{llrrrr}
\toprule
{} &        model &  ap\_mean &  ap\_std &  ra\_mean &  ra\_std \\
\midrule
0 &  tfidf\_words &    0.575 &   0.112 &    0.543 &   0.131 \\
1 &    mlp\_words &    0.654 &   0.127 &    0.650 &   0.135 \\
2 &  tfidf\_chars &    0.674 &   0.105 &    0.634 &   0.131 \\
3 &    mlp\_chars &    0.664 &   0.131 &    0.682 &   0.115 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3888692/2842510611.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results_df.round(3).to_latex())


In [70]:
results_df.to_pickle("quotes_results_classical.pkl")